In [1]:
!pip install wbdata

You are using pip version 9.0.3, however version 20.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import wbdata as wb
import pandas as pd
import numpy as np
import datascience as ds

In [3]:
index_data = np.arange(1959, 2020)
reverse_index = index_data[60:0:-1]

In [4]:
gdp_per_capita_dic = {"NY.GDP.PCAP.CD": "GDP per capita (current US$)"}
gdp_per_capita = wb.get_dataframe(gdp_per_capita_dic, "USA")
gdp_per_capita_data_loc = gdp_per_capita.loc[:,"GDP per capita (current US$)"].values[0:60]

In [5]:
ag_merchandise_exports_percent_dic = {"TX.VAL.AGRI.ZS.UN": "Agricultural raw materials exports (% of merchandise exports)"}
ag_merchandise_exports_percent = wb.get_dataframe(ag_merchandise_exports_percent_dic, "USA")
ag_merchandise_exports_loc = ag_merchandise_exports_percent.loc[:,  "Agricultural raw materials exports (% of merchandise exports)"].values[0:60]
ag_merchandise_exports_loc_percents = ag_merchandise_exports_loc/100
export_percents = ds.make_array(ag_merchandise_exports_percent['Agricultural raw materials exports (% of merchandise exports)'])

tot_merchandise_exports_dic = {"TX.VAL.MRCH.CD.WT": "Merchandise exports (current US$)"}
tot_merchandise_exports = wb.get_dataframe(tot_merchandise_exports_dic, "USA")
tot_merchandise_exports_data = ds.make_array(tot_merchandise_exports['Merchandise exports (current US$)'])
tot_merchandise_exports_loc = tot_merchandise_exports.loc[:, "Merchandise exports (current US$)"].values[0:60]


ag_export_value_tot = ((export_percents/100) * tot_merchandise_exports_data)
ag_export_value_in_millions = ((export_percents/100) * tot_merchandise_exports_data)/1000000

ag_export_value_loc_tot_in_millions = (ag_merchandise_exports_loc_percents*tot_merchandise_exports_loc)/1000000

In [6]:
crop_prod_index_dic = {"AG.PRD.CROP.XD": "Crop production index (2004-2006 = 100)"}
crop_prod_index = wb.get_dataframe(crop_prod_index_dic, "USA")

crop_prod_index_data = ds.make_array(crop_prod_index['Crop production index (2004-2006 = 100)'])
crop_prod_loc = crop_prod_index.loc[:, 'Crop production index (2004-2006 = 100)'].values[0:60]

In [7]:
rural_pop_dic = {"SP.RUR.TOTL": "Rural population"}
rural_pop = wb.get_dataframe(rural_pop_dic, "USA")
rural_loc = rural_pop.loc[:, 'Rural population'].values[0:60]

total_pop_dic = {"SP.POP.TOTL": "Population, total"}
total_pop = wb.get_dataframe(total_pop_dic, "USA")
total_pop_loc = total_pop.loc[:, 'Population, total'].values[0:60]



non_rural_loc_in_millions = (total_pop_loc- rural_loc)/1000000

In [8]:
recession_data = pd.read_csv("USREC.csv")
flipped_recession_data = recession_data.iloc[::-1]
flipped_recession_data[flipped_recession_data['USREC'] > 0] = 1

In [9]:
prelim_df = pd.DataFrame({
    'Year': reverse_index,
    "GDP per Capita": gdp_per_capita_data_loc,
    "Crop Production Index": crop_prod_loc,
    'Agricultural Export Value(in millions of USD)': ag_export_value_loc_tot_in_millions,
    'Non Rural Population (in millions)': non_rural_loc_in_millions,
    "Recession": list(flipped_recession_data['USREC']),
})

prelim_df.tail()

,Year,GDP per Capita,Crop Production Index,Agricultural Export Value(in millions of USD),Non Rural Population (in millions),Recession
55,1964,3573.941185,44.06,1639.453305,137.215986,0.0
56,1963,3374.515171,45.81,1422.241100,134.615404,0.0
57,1962,3243.843078,43.93,1309.425446,131.988693,0.0
58,1961,3066.562869,43.54,NaN,129.276215,1.0
59,1960,3007.123445,NaN,NaN,126.462473,1.0


In [10]:
indiana_weather = pd.read_csv("USC00129222.csv")
framed_indiana_weather = indiana_weather[(indiana_weather['DATE'] > 1959) & (indiana_weather['DATE'] < 2020)] 
indiana_rain_data = framed_indiana_weather['PRCP']
indiana_rain_data_list = indiana_rain_data.tolist()
indiana_rain_data_list_insert = indiana_rain_data_list.insert(0,0)


nc_weather = pd.read_csv('USW00013722.csv')
framed_nc_weather = nc_weather[(nc_weather['DATE'] > 1959) & (nc_weather['DATE'] < 2020)]
nc_rain_data = framed_nc_weather['PRCP']

wyoming_weather = pd.read_csv("USW00024089.csv")
framed_wyoming_weather = wyoming_weather[(wyoming_weather['DATE'] > 1959) & (wyoming_weather['DATE'] < 2020)]
wy_rain_data = framed_wyoming_weather['PRCP']

oregon_weather = pd.read_csv("USC00351862.csv")
framed_oregon_weather = oregon_weather[(oregon_weather['DATE'] > 1959) & (oregon_weather['DATE'] < 2020)]
oregon_rain_data = framed_oregon_weather['PRCP']
oregon_rain_data_list = oregon_rain_data.tolist()
oregon_rain_data_list_insert = oregon_rain_data_list.insert(0,0)


texas_weather = pd.read_csv("USW00013958.csv")
framed_tx_data = texas_weather[(texas_weather['DATE'] > 1959) & (texas_weather['DATE'] < 2020)]
texas_rain_data = framed_tx_data['PRCP']

In [11]:
rain_df = pd.DataFrame({
    "Year" : reverse_index,
    'IN Rain': indiana_rain_data_list,
    'NC Rain': nc_rain_data.values.tolist(),
    'WY Rain' : wy_rain_data.values.tolist(),
    'OR Rain' : oregon_rain_data_list,
    'TX Rain' : texas_rain_data.values.tolist()
},
index = np.arange(0,60))
rain_df['Sampled Rain Average'] = round(rain_df.mean(axis=1),1)
rain_df.tail()

,Year,IN Rain,NC Rain,WY Rain,OR Rain,TX Rain,Sampled Rain Average
55,1964,980.6,1450.6,338.1,1041.8,1523.5,1216.4
56,1963,1076.2,1302.0,396.0,NaN,987.4,1144.9
57,1962,1083.8,1157.1,358.7,NaN,882.2,1088.8
58,1961,1054.8,1531.6,333.4,840.1,1030.0,1125.1
59,1960,1128.1,1091.9,368.7,854.6,809.6,1035.5


In [12]:
prelim_df['Sampled Rain Average'] = rain_df['Sampled Rain Average']
prelim_df.tail()

,Year,GDP per Capita,Crop Production Index,Agricultural Export Value(in millions of USD),Non Rural Population (in millions),Recession,Sampled Rain Average
55,1964,3573.941185,44.06,1639.453305,137.215986,0.0,1216.4
56,1963,3374.515171,45.81,1422.241100,134.615404,0.0,1144.9
57,1962,3243.843078,43.93,1309.425446,131.988693,0.0,1088.8
58,1961,3066.562869,43.54,NaN,129.276215,1.0,1125.1
59,1960,3007.123445,NaN,NaN,126.462473,1.0,1035.5
